In [ ]:
!pip install flask-ngrok
!pip install pyngrok
!ngrok version
!ngrok authtoken ''

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0

!pip install accelerate

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


In [7]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import time
import threading

app = Flask(__name__)
port = "5000"

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model_id = "skitzobaka/Final-Year-Project"
model2 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer2 = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

public_url = ngrok.connect(port).public_url
print(f'* ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

app.config["BASE_URL"] = public_url

@app.route('/', methods=['GET'])
def home():
    return "Hello, world!"

@app.route('/predict_sentiment', methods=['POST'])
def predict_sentiment():
    try:
        data = request.json.get('data', '')
        review = data
        tokens = tokenizer.encode(review, return_tensors='pt')
        with torch.no_grad():
            result = model(tokens)
        sentiment_score = int(torch.argmax(result.logits)) + 1
        senticlass = get_sentiment_class(sentiment_score)
        return jsonify({'sentiment_score': sentiment_score, 'sentiment_class': senticlass})

    except Exception as e:
        return jsonify({'error': str(e)})

@app.route('/recommendations',methods=['POST'])
def recommendations():
  data = request.json.get('data', '')
  prompt=data
  result = get_completion(query=prompt, model=model2, tokenizer=tokenizer2)
  result=result[len(prompt):]
  return jsonify({'recommendations': result})


@app.route('/upload', methods=['POST'])
def upload():
    if 'text_data' not in request.form:
        return jsonify({"error": "No text data provided"}), 400

    text_data = request.form['text_data']
    print("Received text data:", text_data)

    return jsonify({"message": "Text data received and printed successfully"}), 200


def get_sentiment_class(sentiment_score):
    # Define your sentiment classes based on the sentiment score
    # Adjust this function based on your specific requirements
    # Example:
    if sentiment_score == 1:
        return 'Strongly negative'
    elif sentiment_score == 2:
        return 'Mildly negative'
    elif sentiment_score == 3:
        return 'Neutral'
    elif sentiment_score == 4:
        return 'Mildly positive'
    elif sentiment_score == 5:
        return 'Strongly positive'
    else:
        return 'Unable to process please try with a better review'

def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"

  prompt_template = """
  <start_of_turn>user
  Below is an product review that has department_name,class_name,review_text,sentiment_class. Write a response that has recommendations for it.
  {query}
  <end_of_turn>\n<start_of_turn>model
  """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)


  generated_ids = model.generate(**model_inputs, max_new_tokens=300, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  # decoded = tokenizer.batch_decode(generated_ids)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

threading.Thread(target=app.run, kwargs={"use_reloader": False,"debug":True}).start()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

* ngrok tunnel "https://3aed-34-16-171-255.ngrok-free.app" -> "http://127.0.0.1:5000"


In [ ]:
!lsof -i :5000

In [ ]:
!kill 2526